<a href="https://colab.research.google.com/github/YN1752/Spam-Classification/blob/main/Spam_classif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Dataset**

In [1]:
import pandas as pd
import string

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('drive/My Drive/spamdata.csv')
data

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
data['label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: label, dtype: float64

# **Data Pre-processing**

In [8]:
cleaned = data['text'][2].lower()
cleaned

"free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's"

In [6]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
punctuations = string.punctuation

In [18]:
cleaned = "".join(character for character in cleaned if character not in punctuations)
cleaned

'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s'

In [19]:
from spacy.lang.en import English
nlp = English()

In [22]:
my_doc = nlp(cleaned)

In [23]:
token_list = []
for token in my_doc:
  token_list.append(token.text)

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

In [14]:
import random
random.sample(list(data['text']),5)

['I keep ten rs in my shelf:) buy two egg.',
 "K I'm ready,  &lt;#&gt; ?",
 'Have you been practising your curtsey?',
 'I enjoy watching and playing football and basketball. Anything outdoors. And you?',
 "Yes I started to send requests to make it but pain came back so I'm back in bed. Double coins at the factory too. I gotta cash in all my nitros."]

In [25]:
nlp.vocab['free']